In [ ]:
from particle.particle.convert import get_from_pdg_mcd
from particle.particle.utilities import programmatic_name
from particle import data
from particle import PDGID
import numpy as np
import pandas as pd

In [ ]:
mass_width = data.open_text(data, 'mass_width_2019.mcd')
table = get_from_pdg_mcd(mass_width)

del table['Mass'], table['MassUpper'], table['MassLower'], table['Width'], table['WidthUpper'], table['WidthLower']
table.Name = table.Name.apply(programmatic_name)

pp = table.charge == '++'
p = table.charge == '+'
o = table.charge == '0'
m = table.charge == '-'
baryons = table.apply(lambda x: PDGID(x.name).is_baryon, axis=1)

pp_inv = table[pp].copy()
pp_inv.Name = pp_inv.Name + '_minusminus'
pp_inv.charge = '--'
pp_inv.index *= -1

p_inv = table[p].copy()
p_inv.Name.where(~baryons[p], p_inv.Name + '_bar', inplace=True)
p_inv.Name = p_inv.Name + '_minus'
p_inv.charge = '-'
p_inv.index *= -1

o_inv = table[o].copy()
o_inv.Name = o_inv.Name + '_bar_0'
o_inv.charge = '0'
o_inv.index *= -1

m_inv = table[m].copy()
m_inv.Name.where(~baryons[m], m_inv.Name + '_bar', inplace=True)
m_inv.Name = m_inv.Name + '_plus'
m_inv.charge = '+'
m_inv.index *= -1

table.Name.where(~o, table.Name + '_0', inplace=True)
table.Name.where(~m, table.Name + '_minus', inplace=True)
table.Name.where(~p, table.Name + '_plus', inplace=True)
table.Name.where(~pp, table.Name + '_plusplus', inplace=True)

table = pd.concat([table, pp_inv, p_inv, o_inv, m_inv])

In [ ]:
stable = table.sort_values(by='Name')
stable.head()

In [ ]:
len(set(table.Name)), len(table.Name)

In [ ]:
vs = np.asarray(stable.iloc[1:].Name) == np.asarray(stable.iloc[:-1].Name)

In [ ]:
paired = np.concatenate([vs, [False]]) | np.concatenate([[False], vs])

In [ ]:
stable[paired]

In [ ]:
table['tmp'] = abs(table.index - .25)
table.sort_values(by='tmp', inplace=True)
del table['tmp']

In [ ]:
for pdgid, item in table.Name.iteritems():
    print('{item:20}= {pdgid},'.format(item=item, pdgid=pdgid))